In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer ## to creae bag of words

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import math



pd.set_option('display.max_columns', 100)
pd.set_option('display.expand_frame_repr', True)
pd.options.display.max_colwidth = 100000

import warnings
warnings.filterwarnings("ignore")

In [8]:
SentenceTransformer

sentence_transformers.SentenceTransformer.SentenceTransformer

In [11]:
import sentence_transformers
print(nltk.__version__)


3.6.5


In [22]:
df

,id,job_title,location,connection,fit,selected,similarity
0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,0,0
1,2,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,NaN,0,0
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0,0
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN,0,0
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN,0,0
...,...,...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating May 2020 | Seeking an Entry-Level Human Resources Position in St. Louis,"Cape Girardeau, Missouri",103,NaN,0,0
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN,0,0
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,0,0
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN,0,0


In [37]:
talents

,Unnamed: 0,id,job_title,location,connection,fit,selected,similarity
0,0,1,2019 C.T. Bauer College of Business Graduate (Magna Cum Laude) and aspiring Human Resources professional,"Houston, Texas",85,NaN,0,0
1,1,2,Native English Teacher at EPIK (English Program in Korea),Kanada,500,NaN,0,0
2,2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0,0
3,3,4,People Development Coordinator at Ryan,"Denton, Texas",500,NaN,0,0
4,4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500,NaN,0,0
...,...,...,...,...,...,...,...,...
99,99,100,Aspiring Human Resources Manager | Graduating May 2020 | Seeking an Entry-Level Human Resources Position in St. Louis,"Cape Girardeau, Missouri",103,NaN,0,0
100,100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500,NaN,0,0
101,101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN,0,0
102,102,103,Always set them up for Success,Greater Los Angeles Area,500,NaN,0,0


In [70]:
talents2 = talents.copy()

In [71]:
## clean the text and convert it to lower case

def clean_text(job_titles):
    employee_titles = []
    for i in range(0, len(job_titles)):
        titles = re.sub('[^a-zA-Z]', ' ', job_titles[i])
        titles = titles.lower()
        titles = titles.split()
        titles = ' '.join(titles)
        employee_titles.append(titles)
    return employee_titles

In [72]:
##  user to input the job title
key1_titel = input('write the jobe describtion or skils you are loking for: ')
if key1_titel:
    key1_titel = [key1_titel]
else:
    key1_titel = []

write the jobe describtion or skils you are loking for: 


In [114]:
##  user to input the job location
key1_location = input('best location: ')
if key1_location:
    key1_location = [key1_location]
else:
    key1_location = []

best location: İzmir, Türkiye


In [115]:


## define sigmoid function to calculate the probability from similarity
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [116]:
## OpenAI Tranformer language model
model = SentenceTransformer('all-MiniLM-L6-v2')


# lets try to make system work as folowing looking for location similarity and job similarity individualy then combine the results. after that the model will take the selected employee effect into consideration


In [117]:
job_title = talents['job_title']
location = talents['location']

In [118]:
# clean the text
job_title = clean_text(job_title)
location = clean_text(location)


In [119]:
# Embedding the titles
titles_embeddings = model.encode(job_title, show_progress_bar=True)

Batches: 100%|██████████| 4/4 [00:00<00:00,  9.80it/s]


In [120]:
# Embedding the location
location_embeddings = model.encode(location)

In [121]:
# Embedding the titles keys
if key1_titel:
    key_title_embeddings = model.encode(key1_titel)

In [122]:
# Embedding the locations keys
if key1_location:
    key_location_embeddings = model.encode(key1_location)

In [123]:
## lets find the similarities between keys and values, then between employess and the selected one if there is a selected employee
# title similarity 
if key1_titel:
    title_similairt = cosine_similarity(titles_embeddings, key_title_embeddings)
## loation similarity
if key1_location:
    location_similairt = cosine_similarity(location_embeddings, key_location_embeddings)


In [3]:
## all cases to calculate the similarity
employee_number = ''
## all cases to calculate the similarity
def similarity_calculate(employee_num=employee_number, key1_location=key_location, key1_titel=key_titel):
    global employee_number
    
    if not employee_number:
        if not key1_location:
            if talents['similarity'].sum() == 0:
                talents['similarity'] = pd.Series(title_similairt.ravel())
            else:
                talents['similarity'] = (talents['similarity'] + pd.Series(title_similairt.ravel()))/2
        elif not key1_titel:
            if talents['similarity'].sum() == 0:
                talents['similarity'] = pd.Series(location_similairt.ravel())
            else:
                talents['similarity'] =  (talents['similarity'] + pd.Series(location_similairt.ravel()))/2
        else:  
            combin_sim = (location_similairt + title_similairt)/2
            if talents['similarity'].sum() == 0:
                talents['similarity'] = combin_sim
            else:
                talents['similarity'] = (talents['similarity'] + pd.Series(combin_sim.ravel()))/2

        ## lets combine the similarity by averaging the values
        talents['fit'] = [sigmoid(sim) for sim in talents['similarity']]
        return talents.sort_values(by=['fit'], ascending=False).head(15)

    else:
        talents.loc[talents['id'] == int(employee_number), 'selected'] = 1
        ## lets extract the index 
        idx= talents[talents['id'] == int(employee_number)].index[0]
        selecte_title = titles_embeddings[idx]
        selected_location = location_embeddings[idx]

        select_emp_title_sim = cosine_similarity(titles_embeddings, selecte_title.reshape(1,-1))
        select_emp_location_sim = cosine_similarity(location_embeddings, selected_location.reshape(1,-1))

        ### lets average with previous state
        combin_emp_sim = (select_emp_title_sim + select_emp_location_sim)/2
        talents['similarity'] = (0.4*talents['similarity'] +  0.6*pd.Series(combin_emp_sim.ravel()))/2
        ## lets combine the similarity by averaging the values
        talents['fit'] = [sigmoid(sim) for sim in talents['similarity']]

        employee_number = ''
        return talents.sort_values(by=['fit'], ascending=False).head(15)


NameError: name 'key_location' is not defined

In [125]:
similarity_calculate()

,id,job_title,location,connection,fit,selected,similarity
22,23,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0.582180,0,0.331727
34,35,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0.582180,0,0.331727
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0.582180,0,0.331727
47,48,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0.582180,0,0.331727
1,2,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,0.566792,0,0.268776
31,32,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,0.566792,0,0.268776
15,16,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,0.566792,0,0.268776
44,45,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,0.566792,0,0.268776
19,20,Native English Teacher at EPIK (English Program in Korea),Kanada,500+,0.566792,0,0.268776
97,98,Student,"Houston, Texas Area",4,0.563640,0,0.255948


In [85]:
## select an employe
employee_number = input('add employee number to be selectd as top performer: ')


add employee number to be selectd as top performer: 


In [86]:
similarity_calculate()

,id,job_title,location,connection,fit,selected,similarity
75,76,Aspiring Human Resources Professional | Passionate about helping to create an inclusive and engaging work environment,"New York, New York",212,0.572821,0,0.293370
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.564965,0,0.261339
48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,0.564965,0,0.261339
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,0.564965,0,0.261339
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.564965,0,0.261339
59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,0.564965,0,0.261339
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,0.564965,0,0.261339
67,68,Human Resources Specialist at Luxottica,Greater New York City Area,500+,0.564965,0,0.261339
90,91,Lead Official at Western Illinois University,Greater Chicago Area,39,0.557996,0,0.233032
47,48,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,0.551812,0,0.207994


In [1]:
#!/usr/bin/env python

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import math

pd.set_option('display.max_columns', 100)
pd.set_option('display.expand_frame_repr', True)
pd.options.display.max_colwidth = 100000


class JobSearch:

    def __init__(self, file_path= "..\Data", file_name= "potential-talents.csv"):
        self.talents = pd.read_csv(f"{file_path}/{file_name}")
        self.talents2 = self.talents.copy()
        self.model = SentenceTransformer('all-MiniLM-L6-v2')
        self.employee_number = ''



    def clean_text(self, job_titles):
        employee_titles = []
        for i in range(0, len(job_titles)):
            titles = re.sub('[^a-zA-Z]', ' ', job_titles[i])
            titles = titles.lower()
            titles = titles.split()
            titles = ' '.join(titles)
            employee_titles.append(titles)
        return employee_titles

    def get_job_location(self):
        job_location = input('best location: ')
        if job_location:
            return [job_location]
        else:
            return []

    def get_job_titles(self):
        job_titles = input('write the jobe describtion or skils you are loking for: ')
        if job_titles:
            return [job_titles]
        else:
            return []
        
    def set_employee_number(self):
        self.employee_number = input('write the the s: ')
        
    def embedding(self, embedded_vector):
        return self.model.encode(embedded_vector)
    
    def sigmoid(self, x):
        return 1 / (1 + math.exp(-x))
    

    
    def get_similarity(self, data, titles_embeddings, key_title_embeddings, key_title, location_embeddings, key_location_embeddings, key_location):
        

        
        title_similairt = cosine_similarity(titles_embeddings, key_title_embeddings) if key_title else None
        location_similairt = cosine_similarity(location_embeddings, key_location_embeddings) if key_location else None
        employee_number = self.employee_number
        
        if not employee_number:
            if not key_location:
                if data['similarity'].sum() == 0:
                    data['similarity'] = pd.Series(title_similairt.ravel())
                else:
                    data['similarity'] = (data['similarity'] + pd.Series(title_similairt.ravel()))/2
            elif not key_title:
                if data['similarity'].sum() == 0:
                    data['similarity'] = pd.Series(location_similairt.ravel())
                else:
                    data['similarity'] =  (data['similarity'] + pd.Series(location_similairt.ravel()))/2
            else:  
                combin_sim = (location_similairt + title_similairt)/2
                if data['similarity'].sum() == 0:
                    data['similarity'] = combin_sim
                else:
                    data['similarity'] = (data['similarity'] + pd.Series(combin_sim.ravel()))/2

            ## lets combine the similarity by averaging the values
            data['fit'] = [self.sigmoid(sim) for sim in data['similarity']]
            return data.sort_values(by=['fit', 'connection'], ascending=False).head(15)

        else:
            data.loc[data['id'] == int(employee_number), 'selected'] = 1
            ## lets extract the index 
            idx= data[data['id'] == int(employee_number)].index[0]
            selecte_title = titles_embeddings[idx]
            selected_location = location_embeddings[idx]

            select_emp_title_sim = cosine_similarity(titles_embeddings, selecte_title.reshape(1,-1))
            select_emp_location_sim = cosine_similarity(location_embeddings, selected_location.reshape(1,-1))

            ### lets average with previous state
            combin_emp_sim = (select_emp_title_sim + select_emp_location_sim)/2
            data['similarity'] = (0.4*data['similarity'] +  0.6*pd.Series(combin_emp_sim.ravel()))/2
            ## lets combine the similarity by averaging the values
            data['fit'] = [self.sigmoid(sim) for sim in data['similarity']]

            employee_number = ''
            return data.sort_values(by=['fit', 'connection'], ascending=False).head(15)


In [2]:
user_file_path = input("Enter the file path: ")
user_file_name = input("Enter the file name: ")
if not user_file_path:
    search = JobSearch()
else:
    search = JobSearch(user_file_path, user_file_name)

Enter the file path: 
Enter the file name: 


C:\Users\abdullahq.MOLG0\Anaconda3\lib\site-packages\torch\cuda\__init__.py:82: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10\cuda\CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
search = JobSearch()

In [4]:
key_location = search.get_job_location()
key_location_embedding = search.embedding(key_location)

best location: new york


In [5]:
key_title = search.get_job_titles()
key_title_embedding = search.embedding(key_title)

write the jobe describtion or skils you are loking for: human resource


In [6]:
## lets clean and eembidd every thing we have 

In [7]:
job_title = search.clean_text(search.talents["job_title"])
job_title_embedding = search.embedding(job_title)

In [8]:
job_location = search.clean_text(search.talents["location"])
job_location_embedding = search.embedding(job_location)

In [9]:
key_location

['new york']

In [10]:
search.set_employee_number()

write the the s: 


In [11]:
search.get_similarity(data=search.talents, titles_embeddings=job_title_embedding, key_title_embeddings=key_title_embedding, key_title=key_title, 
               location_embeddings=job_location_embedding, key_location_embeddings=key_location_embedding, key_location=key_location)

,Unnamed: 0,id,job_title,location,connection,fit,selected,similarity,conAnection
75,75,76,Aspiring Human Resources Professional | Passionate about helping to create an inclusive and engaging work environment,"New York, New York",212,0.688345,0,0.792395,212
5,5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.671544,0,0.715175,1
23,23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.671544,0,0.715175,1
35,35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,0.671544,0,0.715175,1
48,48,49,Aspiring Human Resources Specialist,Greater New York City Area,1,0.671544,0,0.715175,1
59,59,60,Aspiring Human Resources Specialist,Greater New York City Area,1,0.671544,0,0.715175,1
67,67,68,Human Resources Specialist at Luxottica,Greater New York City Area,500,0.661121,0,0.668294,500
73,73,74,Human Resources Professional,Greater Boston Area,16,0.659383,0,0.660547,16
98,98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.644458,0,0.594766,48
27,27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.642081,0,0.584406,390


In [13]:
d = input('write the the s: ')

write the the s: 15


In [15]:
int(d)

15